In [81]:
import pandas as pd
pd.set_option('display.max_row', 2000)
pd.set_option('display.max_column', 100)
import warnings
warnings.filterwarnings('ignore')

path = "https://raw.githubusercontent.com/Soyoung-Yoon/bigdata/main/"

**9회-1번) 대출액 차이 분석**

은행과 자본 대출 데이터를 활용하여 각 지역에 대해 연도 및 성별별 총대출액을 집계하고, 지역 간 대출 격차를 분석하려 한다.

다음 절차에 따라 문제를 해결하시오.
- 1) 각 지역(Region_Code)에 대해 연도(Year) 및 성별(Gender)별로 총대출액의 합계를 구하시오.
- - > 총대출액은 은행대출(Bank_Loan) + 캐피탈대출(Capital_Loan)으로 계산한다.
- 2) 총대출액의 연도 및 성별별 합계액 차이의 절댓값이 가장 큰 지역을 구해, 지역(Region_Code)을 정수형으로 출력합니다.

In [17]:
df = pd.read_csv(path + "loan_data.csv")
# print(df.head(3))

# 각 지역(Region_Code)에 대해 연도(Year) 및 성별(Gender)별로 총대출액의 합계를 구하시오
df['All_Loan'] = df['Bank_Loan'] + df['Capital_Loan']
# print(df.head(3))
s1 = df.groupby(['Region_Code','Year'], as_index=False)['All_Loan'].sum().rename(columns={'All_Loan':'All_Loan_Year'})
s2 = df.groupby(['Region_Code','Gender'], as_index=False)['All_Loan'].sum().rename(columns={'All_Loan':'All_Loan_Gender'})

# 총대출액의 연도 및 성별별 합계액 차이의 절댓값이 가장 큰 지역을 구해, 
# 지역(Region_Code)을 정수형으로 출력합니다.
s = pd.merge(s1, s2, "left", "Region_Code")
s['Diff'] = (s['All_Loan_Year'] - s['All_Loan_Gender']).abs()
result = int(s.loc[s['Diff'] == s['Diff'].max(), 'Region_Code'].values[0])
# print(result) # 4146510700


4146510700


**9회-2번) 연도별 최고 검거율 범죄 유형 분석**

범죄 발생 및 검거 데이터를 기반으로 범죄유형별 검거율을 계산하고, 각 연도별로 검거율이 가장 높은 범죄유형의 검거 실적을 분석한다.

다음 절차에 따라 문제를 해결하시오.
- 1) 검거율은 다음과 같이 계산한다.
-  > 검거율 = 검거건수 / 발생건수
- 2) 범죄유형별로 연도별 검거율을 계산하시오.
- 3) 각 연도별로 검거율이 가장 높은 범죄유형을 찾으시오.
- 4) 해당 범죄유형의 검거건수를 구하고, 그 값들을 모두 합한 값을 정수형으로 출력하시오.

In [55]:
df = pd.read_csv(path + "crime_data.csv")
# print(df.head(3))
df = df.melt(id_vars=['연도','구분'], value_vars=[col for col in df.columns if '유형' in col],
             var_name='유형', value_name='건수')
df = df.pivot_table(values='건수', columns='구분', index=['연도','유형']).reset_index()

# 범죄유형별로 연도별 검거율을 계산하시오.
df['검거율'] = df['검거건수'] / df['발생건수']
# print(df.head(3))

# 각 연도별로 검거율이 가장 높은 범죄유형을 찾으시오.
# print(df['연도'].unique()) # 2014 2015 2016 2017 2018 2019 2020, int
s1 = df.groupby('연도')['검거율'].max().reset_index().sort_values(by='검거율', ascending=False)
s2 = pd.merge(df, s1, "left", "연도")
# print(s2)

# 해당 범죄유형의 검거건수를 구하고, 그 값들을 모두 합한 값을 정수형으로 출력하시오.
result = int(s2[s2['검거율_x']==s2['검거율_y']]['검거건수'].sum())
# print(result) # 7799
 

**9회-3번) 근속연수 및 교육참가 분석**

사원 데이터의 결측값을 조건에 따라 적절히 처리하고, 부서와 조건별 평균을 활용하여 특정 계산을 수행한다.

다음 절차에 따라 문제를 해결하시오.
- 1) 평균만족도에 결측치가 있는 경우, 전체 평균만족도의 평균값으로 채우시오.
- 2) 근속연수에 결측치가 있는 경우,
같은 부서와 같은 성과등급을 가진 사원들의 근속연수 평균을 정수로 변환하여 채우시오.
- 3) 변수 A는 부서가 'Sales'이고 성과등급이 'C'인 사원들의 평균 근속연수로 정의하시오.
- 4) 변수 B는 부서가 'Operations'이고 평균만족도가 2.5 이상인 사원들의 평균 교육참가횟수로 정의하시오.
- 5) 최종적으로 A + B의 값을 정수로 출력하시오.

In [83]:
df = pd.read_csv(path + "hr_data.csv")
# print(df.head(3))
# print(df.isna().sum().to_frame().T)

# 1. 평균만족도에 결측치가 있는 경우, 전체 평균만족도의 평균값으로 채우시오.
df['평균만족도'] = df['평균만족도'].fillna(df['평균만족도'].mean())
# df.isna().sum().to_frame().T

# 2. 근속연수에 결측치가 있는 경우, 같은 부서와 같은 성과등급을 가진 사원들의 근속연수 평균을 정수로 변환하여 채우시오.
# print(df.isna().sum().to_frame().T)
s = round(df.groupby(['부서','성과등급'])['근속연수'].transform('mean'))
df['근속연수'] = df['근속연수'].fillna(s)
# print(df.isna().sum().to_frame().T)

# 3. 변수 A는 부서가 'Sales'이고 성과등급이 'C'인 사원들의 평균 근속연수로 정의하시오.
A = df[(df['부서'] == 'Sales') & (df['성과등급'] == 'C')]['근속연수'].mean()
# print(A)

# 4.변수 B는 부서가 'Operations'이고 평균만족도가 2.5 이상인 사원들의 평균 교육참가횟수로 정의하시오.
# print(df.head(3))
B = df[(df['부서'] == 'Operations') & (df['평균만족도'] >= 2.5)]['교육참가횟수'].mean()
# print(B)

# 5. 최종적으로 A + B의 값을 정수로 출력하시오.
result = int(A + B)
# print(result) # 20


**17-4) 연도별 최고 고용비중 산업 분석**

산업별 고용 데이터를 활용하여 각 연도별 고용비중이 가장 높은 산업을 분석한다.

다음 절차에 따라 문제를 해결하시오.
- 1) 산업(Industry)별 연도(Year)별 고용 인원을 기반으로, 전년도 대비 고용 증가율을 다음과 같이 계산하시오.
> 고용 증가율 = (이번년도 고용 - 전년도 고용) / 전년도 고용
- 2) 각 연도별로 고용 증가율이 가장 높은 산업을 찾으시오.
- 3) 2)에서 찾은 산업의 고용 증가량을 구하시오.
> 고용 증가량 = (이번년도 고용 - 전년도 고용)
- 4) 연도별로 구한, 고용 증가량을 합산하여 정수형으로 출력하시오.
- 단, 전년도 데이터가 존재하는 연도(2011년 이상)에 대해서만 계산한다.


In [102]:
df = pd.read_csv(path + "employment_data.csv")
# print(df.head(3))

# 1. 산업(Industry)별 연도(Year)별 고용 인원을 기반으로, 전년도 대비 고용 증가율을 다음과 같이 계산하시오.
#    고용 증가율 = (이번년도 고용 - 전년도 고용) / 전년도 고용
thisYear = df.pivot(index='Year', columns='Industry', values='Employment')
# print(thisYear)
lastYear = thisYear.shift(1)
# print(lastYear)

# 2. 각 연도별로 고용 증가율이 가장 높은 산업을 찾으시오.
increased_value = thisYear.diff(1)
increased_rate = (thisYear - lastYear) / lastYear
# print(increased_rate)

# 3. 2)에서 찾은 산업의 고용 증가량을 구하시오.
s = increased_rate.dropna().idxmax(axis=1)
temp = [increased_value.loc[year, industry] for year, industry in zip(s.index, s.values)]
# 4. 연도별로 구한, 고용 증가량을 합산하여 정수형으로 출력하시오.
result = int(sum(temp))
print(result) # 6571138

6688157
